Python version used for pytwoway : 3.10 (3.10.12 or 3.10.13 is fine)

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pytwoway as tw
import bipartitepandas as bpd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import time
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning) # Useful for logistic regression
pd.options.mode.chained_assignment = None  # default='warn' # Remove copy on slice warning

c:\Users\utilisateur\Downloads\G4H_AKM_RS_NRS\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First, there's graph_formation which basically simply simulates a graph.

In [5]:
def graph_formation(n_patients,
                    n_doctors,
                    max_number_connections,
                    z=1.4,
                    type_distance="default",
                    beta_age_p_graph=0.01,
                    beta_age_d_graph=0.01,
                    beta_sex_p_graph=0.5,
                    beta_sex_d_graph=0.5,
                    beta_distance_graph=-0.5,
                    alpha_law_graph=(-1, 1),
                    psi_law_graph=(-1, 1),
                    nb_latent_factors = 1
                   ):
    """
    Creates only the graph formation part and returns the associated dataframe
    """
    coor_patients = []
    coor_doctors = []
    alpha_graph = []
    psi_graph = []
    rng = np.random.default_rng(None)
    D = np.zeros([n_patients, n_doctors], dtype = np.ndarray)

    if nb_latent_factors == None: # We pick a random number of latent factors
        random_number = random.randint(1, 10)
        for i in range(n_patients):
            
            # We generate the FE for the graph formation model
            alpha_graph.append( np.random.uniform(alpha_law_graph[0], alpha_law_graph[1], size = random_number) )

        for j in range(n_doctors):

            # We generate the FE for the graph formation model
            psi_graph.append( np.random.uniform(psi_law_graph[0], psi_law_graph[1], size = random_number ) )

    else:
        
        for i in range(n_patients):
            
            # We generate the FE for the graph formation model
            alpha_graph.append( np.random.uniform(alpha_law_graph[0], alpha_law_graph[1], size = nb_latent_factors) )

        for j in range(n_doctors):

            # We generate the FE for the graph formation model
            psi_graph.append( np.random.uniform(psi_law_graph[0], psi_law_graph[1], size = nb_latent_factors ) )

    # Generate distance matrix
    if type_distance == 'default':
        for i in range(n_patients):
            # Generate the coordinates of the patients
            coor_patients.append( np.random.uniform(0, 1, 2) )
            for j in range(n_doctors):
                if i == 0: # We ensure each coordinate is generated once for each doctor
                    # Generate the coordinates of the doctors
                    coor_doctors.append( np.random.uniform(0, 1, 2) )
                d = np.sqrt(np.power((coor_patients[i][0] - coor_doctors[j][0]), 2) + np.power((coor_patients[i][1] - coor_doctors[j][1]), 2))
                D[i][j] = d

    else:

        # Assign randomly a CODGEO, DEP, or REG to patients and doctors
        dist_matrix = pd.read_csv('../Data/' + type_distance + '.csv')
        
        del dist_matrix[dist_matrix.columns[0]]
        dist_matrix.index = dist_matrix.columns
        for i in range(len(dist_matrix)):
            dist_matrix.iloc[i, i] = 0
        arr = dist_matrix.columns.values
        for i, col in enumerate(arr):
            arr[i] = int(float(arr[i]))
        dist_matrix.columns = arr
        dist_matrix.index = arr

        # Generate code for patient and doctor
        code_patient = []
        code_doctor = []
        for i in range(n_patients):
            random_code = np.random.choice(dist_matrix.columns.values)
            code_patient.append( random_code )
        for j in range(n_doctors):
            random_code = np.random.choice(dist_matrix.columns.values)
            code_doctor.append( random_code )
        for i in range(n_patients):
            for j in range(n_doctors):
                D[i, j] = dist_matrix.loc[code_patient[i], code_doctor[j]]

    D_normed = ( D - D.mean() ) / D.std()
    # Random draws of ages for patients and doctors
    sim_patient_age = rng.integers(low = 1, high = 99, size = n_patients)
    sim_patient_age_normed = ( sim_patient_age - sim_patient_age.mean() ) / sim_patient_age.std()
    sim_doctor_age = rng.integers(low = 26, high = 99, size = n_doctors)
    sim_doctor_age_normed = ( sim_doctor_age - sim_doctor_age.mean() ) / sim_doctor_age.std()

    # Random draws of genders of patients and doctors
    sim_patient_gender = np.random.choice(np.array([1, 2]), n_patients)
    sim_doctor_gender = np.random.choice(np.array([1, 2]), n_doctors)
    # sim_patient_gender = rng.integers(low = 1, high = 3, size = n_patients)
    # sim_doctor_gender = rng.integers(low = 1, high = 3, size = n_doctors)
    sim_patient_gender_normed = ( sim_patient_gender - sim_patient_gender.mean() ) / sim_patient_gender.std()
    sim_doctor_gender_normed = ( sim_doctor_gender - sim_doctor_gender.mean() ) / sim_doctor_gender.std()

    # Bias for patients and doctors
    # bias_patient = np.random.normal(0, 1, size = n_patients)
    # bias_doctor = np.random.normal(0, 1, size = n_doctors)

    # Compile ids
    id_p = np.repeat(range(n_patients), n_doctors)
    id_d = np.tile(range(n_doctors), n_patients)

    # Compile observed features
    age_p_data = np.repeat(sim_patient_age, n_doctors)
    age_d_data = np.tile(sim_doctor_age, n_patients)
    sex_p_data = np.repeat(sim_patient_gender, n_doctors)
    sex_d_data = np.tile(sim_doctor_gender, n_patients)
    # bias_patient_data = np.repeat(bias_patient, n_doctors)
    # bias_doctor_data = np.tile(bias_doctor, n_patients)
    ef_patient_data = np.repeat(alpha_graph, n_doctors)
    ef_doctor_data = np.tile(psi_graph, n_patients)
    if type_distance != 'default':
        code_patient_data = np.repeat(code_patient, n_doctors)
        code_doctor_data = np.tile(code_doctor, n_patients)
    # # P is the matrix with all the connection probabilities
    # P = np.zeros((n_patients, n_doctors))
    # Generate the identifier matrix A based on the distance
    A = np.zeros([n_patients, n_doctors], dtype = np.ndarray)
    for i in range(n_patients):
        for j in range(n_doctors):
            if D[i][j] <= z: # if patient i and doctor j are too far away, there is no relation
                T = np.dot(alpha_graph[i], psi_graph[j]) + beta_age_p_graph * sim_patient_age_normed[i] + beta_age_d_graph * sim_doctor_age_normed[j] \
                + beta_sex_p_graph * sim_patient_gender_normed[i] + beta_sex_d_graph * sim_doctor_gender_normed[j] + beta_distance_graph * D[i][j]
                p = 1 / (1 + np.exp(-T))
                # P[i][j] = p
                A[i][j] = np.random.binomial(1, p)
    
    # Compile relations between doctors and patients
    relation = A.flatten()

    # Merge all columns into a dataframe
    dataframe = pd.DataFrame(data={'i': id_p, 'j': id_d, 'y' : relation, 'age_p': age_p_data, 'age_d': age_d_data, 
                           'sex_p': sex_p_data, 'sex_d': sex_d_data
                            })
    dataframe['distance'] = D[dataframe['i'], dataframe['j']].astype(float)

        # Now, we bound the number of connections (1 <= connections <= max_number_connections)
    # First, we detect the patients who have 0 connection.
    number_of_connections = dataframe.groupby('i').agg({'y': 'sum'})
    zero_connection = number_of_connections[number_of_connections['y'] == 0].index
    for patient in zero_connection:
        # If patient has zero connection, we connect him with the nearest doctor (even if the threshold z isn't respected)
        min_index = dataframe[dataframe['i'] == patient]['distance'].idxmin()
        doctor_to_connect = dataframe.loc[min_index, 'j']
        dataframe.loc[(dataframe['i'] == patient) & (dataframe['j'] == doctor_to_connect), 'y'] = 1
    
    # Then, we detect the patients who have more than max_number_connections. We choose the remaining connections by doctor's popularities (possible to choose randomly).
    number_of_connections = dataframe.groupby('i').agg({'y': 'sum'})
    too_much_connection = number_of_connections[number_of_connections['y'] > max_number_connections].index
    for patient in too_much_connection:
        # We keep the connections with the most popular doctors
        patient_df = dataframe[dataframe['i'] == patient]
        connected_doctors = patient_df[patient_df['y'] == 1]['j'].values
        most_popular_doctors = dataframe[dataframe['j'].isin(connected_doctors)].groupby('j').agg({'y': 'sum'}).sort_values('y', ascending=False)
        not_kept_doctors = most_popular_doctors.index[max_number_connections:].values
        for doctor in not_kept_doctors:
            dataframe.loc[(dataframe['i'] == patient) & (dataframe['j'] == doctor), 'y'] = 0

        # # We keep the connections choosing randomly
        # patient_df = dataframe[dataframe['i'] == patient]
        # connected_doctors = patient_df[patient_df['y'] == 1]['j'].values
        # not_kept_doctors = np.random.choice(connected_doctors, size=len(connected_doctors) - max_number_connections, replace=False)
        # for doctor in not_kept_doctors:
        #     dataframe[(dataframe['i'] == patient) & (dataframe['j'] == doctor)]['y'] = 0

    # Create a dataframe of fixed effects for patients and doctors, then concatenate it with all the data
    k = len(alpha_graph[0]) # Number of latent factors
    ef_patient = pd.DataFrame(np.zeros((n_patients*n_doctors, k)))
    ef_doctor = pd.DataFrame(np.zeros((n_patients*n_doctors, k)))
    for i in range(k):
        ef_patient_element = []
        ef_doctor_element = []
        ef_patient.rename(columns = {i :f'ef_p_{i}'}, inplace = True)
        ef_doctor.rename(columns = {i :f'ef_d_{i}'}, inplace = True)
        for j in range(n_patients):
            
            ef_patient_element += list(np.repeat(alpha_graph[j][i], n_doctors))

        for j in range(n_doctors):
            ef_doctor_element.append(psi_graph[j][i])
        
        ef_patient[f'ef_p_{i}'] = ef_patient_element
        ef_doctor[f'ef_d_{i}'] = np.tile(ef_doctor_element, n_patients)
    dataframe = pd.concat([dataframe, ef_patient, ef_doctor], axis = 1)
    dataframe = dataframe.reset_index().drop(['index'], axis = 1)
    return dataframe

In [6]:
df = graph_formation(n_patients=100,
                     n_doctors=10,
                     max_number_connections=3)

Then, cost_prescription computes the cost prescriptions of the graph created.

In [15]:
def cost_prescription  (df,
                        z=np.sqrt(2),
                        alpha_law_cost=(0, 0.5),
                        psi_law_cost=(0, 0.5),
                        beta_age_p_cost=0.5,
                        beta_age_d_cost=0.5,
                        beta_sex_p_cost=0.5,
                        beta_sex_d_cost=0.5,
                        beta_distance_cost=0.5):

        # drop the rows if there is no relation between patient_i and doctor_j
        patients = df['i'].unique()
        doctors = df['j'].unique()
        dataframe = df.copy()
        dataframe = dataframe.drop(dataframe[dataframe['y'] == 0].index)
        dataframe = dataframe.drop(['y', 'ef_p_0', 'ef_d_0'], axis = 1)
        dataframe = dataframe.reset_index().drop(['index'], axis = 1)

        alpha_cost = {} # These are dicts to use the function map later
        psi_cost = {}

        for i in patients:
        
            # We generate the FE for the cost model
            alpha_cost[i] = np.random.uniform(alpha_law_cost[0], alpha_law_cost[1])
                               
        for j in doctors:

            # We generate the FE for the cost model
            psi_cost[j] = np.random.uniform(psi_law_cost[0], psi_law_cost[1])

        dataframe['cost_EF_patient'] = dataframe['i'].map(alpha_cost).astype(float)
        dataframe['cost_EF_doctor'] = dataframe['j'].map(psi_cost).astype(float)
        # dataframe['distance'] = D[dataframe['i'], dataframe['j']].astype(float)

        # Compute the cost
        dataframe['y'] = dataframe['cost_EF_patient'] + dataframe['cost_EF_doctor'] + beta_age_p_cost * dataframe['age_p'] + beta_age_d_cost * dataframe['age_d'] + beta_sex_p_cost * dataframe['sex_p'] + beta_sex_d_cost * dataframe['sex_d'] + beta_distance_cost * dataframe['distance']
        cost_column = dataframe.pop('y')
        dataframe.insert(2, 'y', cost_column)

        return dataframe

In [16]:
df_cost = cost_prescription(df)

In [17]:
df_cost.head()

,i,j,y,age_p,age_d,sex_p,sex_d,distance,cost_EF_patient,cost_EF_doctor
0,0,1,81.674690,68,91,1,2,0.510266,0.315892,0.103665
1,0,6,78.078768,68,84,1,2,0.168251,0.315892,0.178750
2,0,8,81.873062,68,91,1,2,0.672323,0.315892,0.221008
3,1,1,62.397317,30,91,1,2,0.390559,0.098373,0.103665
4,1,5,50.728433,30,68,1,2,0.186577,0.098373,0.036772


Here's the part about estimation using linear regression.

In [18]:
def cost_estimate(df):
    # We estimate the FE for the cost model using LinearRegression

    model = LinearRegression()
    patients = df['i'].unique()
    doctors = df['j'].unique()
    n_patients = len(patients)
    n_doctors = len(doctors)

    # Add dummy variables
    e_i_cost = pd.DataFrame(np.zeros((len(df), n_patients), dtype=int))
    for i,patient in enumerate(patients):
        e_i_cost.rename(columns = {i :f'p_{patient}'}, inplace = True)
        
    e_j_cost = pd.DataFrame(np.zeros((len(df), n_doctors), dtype=int))
    for j,doctor in enumerate(doctors):
        e_j_cost.rename(columns = {j :f'd_{doctor}'}, inplace = True)
    
    df2 = pd.concat([df, e_i_cost, e_j_cost], axis = 1)
    
    for i in patients:
        indexes = df2[df2['i'] == i].index
        df2[f'p_{i}'][indexes] = [1 for i in range(len(indexes))]
    
    for j in doctors:
        indexes = df2[df2['j'] == j].index
        df2[f'd_{j}'][indexes] = [1 for i in range(len(indexes))]

    # Scale specific columns
    df2['age_p'] =( df['age_p'] - df['age_p'].mean() ) / df['age_p'].std()
    df2['age_d'] =( df['age_d'] - df['age_d'].mean() ) / df['age_d'].std()
    
    y = df2['y'].astype(int)
    X = df2.drop(['i', 'j', 'y', 'cost_EF_patient', 'cost_EF_doctor'], axis = 1)
    
    regression = model.fit(X, y)
    estimates = regression.coef_

    return estimates

In [19]:
estimates = cost_estimate(df_cost)
estimates

array([ 3.80415307e+11, -1.02557211e+12,  3.54337560e+12, -2.14141718e+12,
        3.54370117e-01,  1.37074004e+11,  6.73584492e+11,  1.01243322e+12,
       -3.05333417e+12,  3.20617066e+11,  1.79430096e+11, -3.58984465e+12,
        7.01821887e+11, -3.15216505e+12,  9.70077131e+11, -3.19452114e+12,
       -2.64389195e+12, -3.08157156e+12,  8.43008857e+11, -1.73537331e+11,
       -3.77338772e+12, -3.20863983e+12, -3.29335202e+12,  8.05992161e+10,
        1.06890801e+12,  8.28890160e+11,  2.07667490e+11, -1.73537331e+11,
       -1.59418634e+11,  8.28890160e+11,  6.59465795e+11,  6.64805191e+10,
       -3.01097807e+12,  8.71246251e+11, -2.50270498e+12, -2.71448544e+12,
        8.57127554e+11,  3.82431250e+10,  8.57127554e+11,  1.65311398e+11,
       -8.88251485e+10,  9.41839737e+11,  3.06498369e+11, -2.46034889e+12,
        7.72415372e+11, -2.65801065e+12, -3.63220075e+12, -3.23503603e+10,
        9.84195828e+11,  1.05478931e+12, -2.98274068e+12, -3.49101377e+12,
        4.47685340e+11,  

Estimation using pytwoway. You might want to use 3.10.12 Python version in order to install pytwoway as there are problems installing it using latest Python versions (see https://github.com/arthursabre/Stage-2A/blob/main/requirements.txt for all the versions of my packages used during the implementation).

In [20]:
def cost_estimate_pytwoway(df,
                           type_distance='default',
                           preconditioner='ichol'):

    fecontrol_params = tw.fecontrol_params(
    {
        'ho': True,
        'he': False,
        'feonly': True,
        'continuous_controls': ['distance', 'age_d', 'age_p'],
        'categorical_controls': ['sex_p', 'sex_d'],
        'attach_fe_estimates': True,
        'ncore': 8,
        'preconditioner': preconditioner # It looks like it gives better results (especially for large datasets ?)
    }
    )

    clean_params = bpd.clean_params(
    {
        'connectedness': 'leave_out_spell',
        'collapse_at_connectedness_measure': True,
        'drop_single_stayers': True,
        'drop_returns': 'returners',
        'copy': False
    }
    )

    dataframe = df.copy()
    # Scale specific columns
    dataframe['age_p'] =( dataframe['age_p'] - dataframe['age_p'].mean() ) / dataframe['age_p'].std()
    dataframe['age_d'] =( dataframe['age_d'] - dataframe['age_d'].mean() ) / dataframe['age_d'].std()
     # Change dtype of categorical variables
    dataframe['sex_p'] = dataframe['sex_p'].astype("category")
    dataframe['sex_d'] = dataframe['sex_d'].astype("category")
    # We create a BipartiteDataFrame in order to estimate the FE for the cost model
    if type_distance == 'default':
        
        bdf = bpd.BipartiteDataFrame(dataframe.drop(['cost_EF_patient', 'cost_EF_doctor'] , axis = 1),
                                    custom_categorical_dict = {'sex_p': True,
                                                            'sex_d': True},
                                    custom_dtype_dict = {'sex_p': 'categorical',
                                                        'sex_d': 'categorical'},
                                    custom_how_collapse_dict = {'sex_p': 'first',
                                                                'sex_d': 'first'}) # We transform the dataframe as BipartitePandas dataframe to Estimate the FE.
    else:
        
        bdf = bpd.BipartiteDataFrame(dataframe.drop(['cost_EF_patient', 'cost_EF_doctor', 'code_patient', 'code_doctor'] , axis = 1),
                                    custom_categorical_dict = {'sex_p': True,
                                                            'sex_d': True},
                                    custom_dtype_dict = {'sex_p': 'categorical',
                                                        'sex_d': 'categorical'},
                                    custom_how_collapse_dict = {'sex_p': 'first',
                                                                'sex_d': 'first'}) # We transform the dataframe as BipartitePandas dataframe to Estimate the FE.


    bdf.clean(clean_params)
    fe_estimator = tw.FEControlEstimator(bdf, fecontrol_params)
    fe_estimator.fit()

    return fe_estimator.gamma_hat_dict # Estimates of the EF, Beta for the cost model